In [1]:
import sys
import datetime
import math
import numpy as np
import random

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

In [2]:
def read_tsp_file(file_name):
    file1 = open(file_name, 'r')  # open the file to [r]ead
    lines = file1.readlines()  # add file line by line to lines list
    file1.close()  # close the file

    location_count = int(lines[0])
    counter = 1
    locations = []

    while counter < len(lines) and lines[counter] is not None:
        current_line = lines[counter]
        parts = current_line.split(",")
        coordinate_x = float(parts[1])
        coordinate_y = float(parts[2])
        new_tuple = (coordinate_x, coordinate_y)
        locations.append(new_tuple)
        counter += 1

    return locations

In [3]:
def calculate_distance(loc1, loc2):
    return math.sqrt(math.pow(loc1[0] - loc2[0], 2) + math.pow(loc1[1] - loc2[1], 2))

In [4]:
def create_distance_matrix(locations):
    table = np.zeros((len(locations), len(locations)))
    for i in range(0, len(locations)):
        for j in range(0, len(locations)):
            table[i, j] = calculate_distance(locations[i], locations[j])
    return table

In [5]:
def nearest_neighbour_algorithm(distance_matrix):

    max_distance = np.max(distance_matrix)  # find maximum distance between any two locations

    list_length = len(distance_matrix)  # find the number of locations
    city_list = list(range(0, list_length))
    city_list_out = []  # list of cities the algorithm will return
    total_cost_out = 0
    modified_distance_matrix = np.copy(distance_matrix)  # copy of the distance matrix we will modify

    first_city = random.randrange(0, list_length)  # randomly select the first city
    city_list.remove(first_city)  # remove it from the cities not visited yet list
    city_list_out.append(first_city)  # add first city to the list of cities algorithm will return
    current_city = first_city  # set first city as the current city
    modified_distance_matrix[:, current_city] = 2 * max_distance * np.ones(list_length)  # modify the distance matrix

    while len(city_list) != 0:
        nearest_next_city = np.argmin(modified_distance_matrix[current_city, :])  # find the index of the next city
        total_cost_out += distance_matrix[current_city, nearest_next_city] # update the cost
        city_list.remove(nearest_next_city)  # update things (same as before)
        city_list_out.append(nearest_next_city)
        current_city = nearest_next_city
        modified_distance_matrix[:, current_city] = 2 * max_distance * np.ones(list_length)

    total_cost_out += distance_matrix[current_city, first_city]

    return total_cost_out, city_list_out  # this returns a tuple where item 0 is the cost and item 1 is the tour

In [7]:
def solve_tsp(file_name, max_iter):
    city_locations = read_tsp_file(file_name)
    distance_table = create_distance_matrix(city_locations)

    file1 = open("output.txt", 'w')  # open a file to write

    best_solution = nearest_neighbour_algorithm(distance_table)  # initialise best_solution (with a feasible solution)

    for i in range(0, max_iter):
        current_solution = nearest_neighbour_algorithm(distance_table)
        if current_solution[0] < best_solution[0]:
            best_solution = current_solution
        file1.write(str(current_solution[0]) + " " + str(best_solution[0]) + "\n")

    file1.write(str(best_solution))  # write the best solution and its objective function to the output file
    file1.close()  # close the output file

    return best_solution

In [8]:
def solve_tsp2(file_name, max_iter, max_iter2):  # constructive algorithm and local search algorithm for tsp
    random.seed(10)
    city_locations = read_tsp_file(file_name)
    distance_table = create_distance_matrix(city_locations)

    file1 = open("output.txt", 'w')  # open a file to write

    best_solution = nearest_neighbour_algorithm(distance_table)  # initialise best_solution (with a feasible solution)

    for i in range(0, max_iter):
        current_solution = nearest_neighbour_algorithm(distance_table)  # set current answer
        if current_solution[0] < best_solution[0]:
            best_solution = current_solution  # update the best solution if current answer is better
        file1.write(str(current_solution[0]) + " " + str(best_solution[0]) + "\n")
        # write current solution objective function value to the file
        for j in range(0, max_iter2):
            new_solution = move_one_city_to_best_location(current_solution, distance_table)
            if new_solution[0] < current_solution[0]:  # if new solution is better than current
                current_solution = new_solution  # move your current solution to new solution
                if new_solution[0] < best_solution[0]:  # if new solution is better than new solution
                    best_solution = new_solution  # update the best solution if the new solution is better
            file1.write(str(j) + " " + str(new_solution[0]) + " " + str(best_solution[0]) + "\n")
            # write new and best solution objective function values

    file1.write(str(best_solution)) # write the best solution and its objective function to the output file
    file1.close()  # close the output file

    return best_solution

In [9]:
def tsp_local_search_2(tsp_solution, distance_matrix):
    # fill this function with a local search algorithm for TSP
    return tsp_solution  # return the solution found (it returns what it receives, current_solution, right now)

In [10]:
def solve_tsp4(file_name, max_iter, max_iter2): # variable neighbourhood search
    random.seed(10)
    city_locations = read_tsp_file(file_name)
    distance_table = create_distance_matrix(city_locations)

    file1 = open("output.txt", 'w')  # open a file to write

    best_solution = nearest_neighbour_algorithm(distance_table)  # initialise best_solution (with a feasible solution)
    sel_prob = 1 / 3 * np.ones(3)  # initial selection probabilities (neighbourhood algorithms) are equally distributed

    for i in range(0, max_iter):
        current_solution = nearest_neighbour_algorithm(distance_table)
        if current_solution[0] < best_solution[0]:
            best_solution = current_solution
        file1.write(str(current_solution[0]) + " " + str(best_solution[0]) + "\n")
        for j in range(0, max_iter2):
            alg_prob = random.random()  # random number to select the local search algorithm
            if alg_prob < sel_prob[0]:  # if the probability is lower than first local search selection probability
                new_solution = move_one_city_to_best_location(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:  # if new solution is better than current solution
                    current_solution = new_solution  # update current solution
                    if new_solution[0] < best_solution[0]: # if the new solution is better than best solution
                        best_solution = new_solution  # update the best solution
                        sel_prob[1] *= 0.9  # decrease probability of other neighbourhoods to 90%
                        sel_prob[2] *= 0.9  # decrease probability of other neighbourhoods to 90%
                        sel_prob[0] = 1 - (sel_prob[1] + sel_prob[2])  # update probability of successful neighbourhood
            elif alg_prob < (sel_prob[0] + sel_prob[1]): # if the alg_prob. is between the zeroth and first sel_prob
                new_solution = tsp_local_search_1(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:
                    current_solution = new_solution
                    if new_solution[0] < best_solution[0]:
                        best_solution = new_solution
                        sel_prob[0] *= 0.9
                        sel_prob[2] *= 0.9
                        sel_prob[1] = 1 - (sel_prob[0] + sel_prob[2])
            else:  # if the first two neighbourhoods were not selected
                new_solution = tsp_local_search_2(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:
                    current_solution = new_solution
                    if new_solution[0] < best_solution[0]:
                        best_solution = new_solution
                        sel_prob[0] *= 0.9
                        sel_prob[1] *= 0.9
                        sel_prob[2] = 1 - (sel_prob[0] + sel_prob[1])

            file1.write(str(j) + " " + str(new_solution[0]) + " " + str(best_solution[0]) + "\n")

    file1.write(str(best_solution))  # write the best solution and its objective function to the output file
    file1.close()  # close the output file

    return best_solution

In [11]:
def solve_tsp5(file_name, max_iter, max_iter2):  # variable neighbourhood search with simulated annealing
    random.seed(10)
    city_locations = read_tsp_file(file_name)
    distance_table = create_distance_matrix(city_locations)

    file1 = open("output.txt", 'w')  # open a file to write

    best_solution = nearest_neighbour_algorithm(distance_table)  # initialise best_solution (with a feasible solution)
    sel_prob = 1 / 3 * np.ones(3)

    for i in range(0, max_iter):
        current_solution = nearest_neighbour_algorithm(distance_table)
        if current_solution[0] < best_solution[0]:
            best_solution = current_solution
        file1.write(str(current_solution[0]) + " " + str(best_solution[0]) + "\n")

        t = 1000  # initialise temperature (simulated annealing)
        alpha = 0.99  # set the cooling factor (simulated annealing)

        for j in range(0, max_iter2):
            alg_prob = random.random()
            if alg_prob < sel_prob[0]:
                new_solution = move_one_city_to_best_location(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:
                    current_solution = new_solution
                    if new_solution[0] < best_solution[0]:
                        best_solution = new_solution
                        sel_prob[1] *= 0.9  # decrease probability of other neighbourhoods to 90%
                        sel_prob[2] *= 0.9  # decrease probability of other neighbourhoods to 90%
                        sel_prob[0] = 1 - (sel_prob[1] + sel_prob[2])  # update probability of successful neighbourhood
                elif math.exp(-(new_solution[0] - current_solution[0]) / t) > random.random():
                    current_solution = new_solution  # current_solution is updated with new one (simulated annealing)
            elif alg_prob < (sel_prob[0] + sel_prob[1]):
                new_solution = tsp_local_search_1(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:
                    current_solution = new_solution
                    if new_solution[0] < best_solution[0]:
                        best_solution = new_solution
                        sel_prob[0] *= 0.9
                        sel_prob[2] *= 0.9
                        sel_prob[1] = 1 - (sel_prob[0] + sel_prob[2])
                elif math.exp(-(new_solution[0] - current_solution[0]) / t) > random.random():
                    current_solution = new_solution  # current solution is updated with the new one (simulated annealing)
            else:
                new_solution = tsp_local_search_2(current_solution, distance_table)
                if new_solution[0] < current_solution[0]:
                    current_solution = new_solution
                    if new_solution[0] < best_solution[0]:
                        best_solution = current_solution
                        sel_prob[0] *= 0.9
                        sel_prob[1] *= 0.9
                        sel_prob[2] = 1 - (sel_prob[0] + sel_prob[1])
                elif math.exp(-(current_solution[0] - best_solution[0]) / t) > random.random():
                    current_solution = new_solution # current solution is updated with the new one (simulated annealing)

            t = t * alpha # update the temperature (simulated annealing)
            file1.write(str(j) + " " + str(new_solution[0]) + " " + str(best_solution[0]) + "\n")

    file1.write(str(best_solution))  # write the best solution and its objective function to the output file
    file1.close()  # close the output file

    return best_solution

In [12]:
def move_one_city(tsp_solution, distance_matrix):
    city_list_current = tsp_solution[1]

    list_length = len(city_list_current)  # find the number of locations
    selected_city_location = random.randrange(0, list_length)  # randomly select a location
    selected_city = tsp_solution[1][selected_city_location]  # find the city that will be moved
    new_location = random.randrange(0, list_length)  # randomly select a new location
    city_list_out = city_list_current.copy()  # copy list of cities from the old solution to the new solution
    city_list_out.remove(selected_city)  # remove the city that is selected randomly
    city_list_out.insert(new_location, selected_city)  # add the city to the place that is selected randomly

    cost_out = 0
    for i in range(0, list_length-1):
        cost_out += distance_matrix[city_list_out[i], city_list_out[i+1]]
    cost_out += distance_matrix[city_list_out[list_length-1],city_list_out[0]]  # calculate the cost from scratch
    return cost_out, city_list_out

In [13]:
def move_one_city_better(tsp_solution, distance_matrix):
    city_list_current = tsp_solution[1]
    cost_current = tsp_solution[0]

    list_length = len(city_list_current)  # find the number of locations
    selected_city_location = random.randrange(0, list_length)  # randomly select a location
    selected_city = tsp_solution[1][selected_city_location]  # find the city that will be moved

    cost_dif = 0  # initialise

    #  city_before -> selected_city -> city_after becomes city_before -> city_after
    #  we need to subtract costs city_before -> selected_city and selected city -> city_after
    #  and add city_before -> city_after

    city_before = city_list_current[selected_city_location-1] # even negative python handles
    city_after = city_list_current[(selected_city_location + 1) % list_length]  # new index within range
    cost_dif -= distance_matrix[city_before, selected_city]  # city_before -> selected_city
    cost_dif -= distance_matrix[selected_city, city_after]  # selected_city -> city_after
    cost_dif += distance_matrix[city_before, city_after]  # city_before -> city_after

    city_list_out = tsp_solution[1].copy()
    city_list_out.remove(selected_city)

    new_location = random.randrange(0, list_length-1)  # randomly select a new location

    # city_before2 -> city_after2 becomes city_before2 -> selected_city -> city_after2
    # we need to add costs city_before2 -> selected_city and selected_city -> city_after2
    # we need to subtract cost city_before2 -> city_after2

    city_before2 = city_list_out[new_location-1]
    city_after2 = city_list_out[new_location]
    cost_dif += distance_matrix[city_before2, selected_city]  # city_before2 -> selected_city
    cost_dif += distance_matrix[selected_city, city_after2]  # selected_city -> city_after2
    cost_dif -= distance_matrix[city_before2, city_after2]  # city_before2 -> city_after2

    cost_out = cost_current + cost_dif
    city_list_out.insert(new_location, selected_city)

    return cost_out, city_list_out

In [14]:
def move_one_city_to_best_location(tsp_solution, distance_matrix):
    city_list_current = tsp_solution[1]
    cost_current = tsp_solution[0]

    list_length = len(city_list_current)  # find the number of locations
    selected_city_location = random.randrange(0, list_length)  # randomly select a location
    selected_city = tsp_solution[1][selected_city_location]  # find the city that will be moved

    cost_dif = 0  # initialise

    city_before = city_list_current[selected_city_location-1] # even negative python handles
    city_after = city_list_current[(selected_city_location + 1) % list_length]  # new index within range
    cost_dif -= distance_matrix[city_before, selected_city]  # city_before -> selected_city
    cost_dif -= distance_matrix[selected_city, city_after]  # selected_city -> city_after
    cost_dif += distance_matrix[city_before, city_after]  # city_before -> city_after

    city_list_out = tsp_solution[1].copy()  # city list to return
    city_list_out.remove(selected_city)  # remove randomly selected city from this list

    best_location = -1
    best_cost_dif = sys.maxsize
    for i in range(0, list_length-1):  # iterate over all possible locations
        city_before2 = city_list_out[i - 1]  # if the city is inserted to location i, city before
        city_after2 = city_list_out[i]  # if the city is inserted to location i, city after

        current_cost_dif = 0  # this variable shows how much cost difference does inserting to current location makes
        current_cost_dif += distance_matrix[city_before2, selected_city]  # city_before2 -> selected_city
        current_cost_dif += distance_matrix[selected_city, city_after2]  # selected_city -> city_after2
        current_cost_dif -= distance_matrix[city_before2, city_after2]  # city_before2 -> city_after2

        if current_cost_dif < best_cost_dif:  # if current is better (smaller), update
            best_cost_dif = current_cost_dif
            best_location = i

    cost_out = cost_current + cost_dif + best_cost_dif  # use the best solution to update
    city_list_out.insert(best_location, selected_city)  # use the best location to update

    return cost_out, city_list_out